In [4]:
import requests
import pandas as pd
from io import StringIO
import geopandas as gpd
import datetime

baseurl = "https://beachwatch.waterboards.ca.gov/public/"
reports_page=f"{baseurl}result.php"
exports_page=f"{baseurl}export.php"

formdata={
   "County":10,
"stationID":"",
"parameter":"",
"qualifier":"",
"method":"",
"created":"",
"year":2025,
"sort":"`SampleDate`",
"sortOrder":"DESC",
"submit":"Search ",
}

limits=[
    {'method':'ddPCR', 'limits':[{'level':'close', 'min':1413,}],},
     {'method':'MF', 'limits':[{'level':'close', 'min':104,}],},
     {'method':'MTF', 'limits':[{'level':'close', 'min':104,}],},
{'method':'Enterolert', 'limits':[{'level':'close', 'min':104,}],}]

In [6]:
response = requests.post(reports_page, data=formdata)
cookies = response.cookies

response = requests.get(exports_page, cookies=cookies)   
data = response.text

beach_df = pd.read_csv(StringIO(data), sep="\t")
#beach_df['date']=  pd.to_datetime(f'{beach_df["SampleDate"]}T{beach_df["SampleTime"]}')#, format='%Y-%m-%d %H:%M:%S')
beach_df['date']=  pd.to_datetime(beach_df["SampleDate"], format='%Y-%m-%d')
beach_df['time']=pd.to_timedelta(beach_df['SampleTime'])
beach_df['datetime']=beach_df['date']+beach_df['time']
gs=gpd.GeoSeries.from_xy(beach_df['Longitude'],beach_df['Latitude'])
beach_gdf= gpd.GeoDataFrame(beach_df, 
    geometry=gs ,
    crs = 'EPSG:4326')

beach_gdf
beach_gdf['Icon']= 'place'

ValueError: expected hh:mm:ss format

In [3]:
columns = beach_gdf.columns.tolist()
[col for col in beach_gdf.columns if pd.api.types.is_datetime64_any_dtype(beach_gdf[col])]

['date', 'datetime']

In [4]:
def fix_col_types(df, date_format=None):
    columns = [col for col in df.columns if pd.api.types.is_datetime64_any_dtype(df[col])]
    for col in columns:
        df[col] = df[col].apply(lambda x: x.strftime(date_format) if pd.notnull(x) and date_format else x.isoformat() if pd.notnull(x) else None)
    return df 
fix_col_types(beach_gdf)
            

,id,Station_ID,Station Name,SampleDate,SampleTime,parameter,qualifier,Result,unit,method,...,Description,Beach Name,Latitude,Longitude,CreateDate,date,time,datetime,geometry,Icon
0,1038,1038,IB-079,2025-03-31,07:27:00,Enterococcus,=,1560.0,Copies/100ml,ddPCR,...,Avenida Lunar,Coronado City beaches,32.674066,-117.172300,2025-04-01 07:15:58,2025-03-31T00:00:00,0 days 07:27:00,2025-03-31T07:27:00,POINT (-117.1723 32.67407),place
1,325,325,EH-050,2025-03-31,07:46:00,Enterococcus,=,941.0,Copies/100ml,ddPCR,...,Loma Ave (frmrly Isabella,Coronado City beaches,32.681840,-117.185260,2025-04-01 07:15:58,2025-03-31T00:00:00,0 days 07:46:00,2025-03-31T07:46:00,POINT (-117.18526 32.68184),place
2,328,328,EH-060,2025-03-31,08:04:00,Enterococcus,=,1320.0,Copies/100ml,ddPCR,...,Navy Fence (A),Coronado north beach,32.685586,-117.194319,2025-04-01 07:15:59,2025-03-31T00:00:00,0 days 08:04:00,2025-03-31T08:04:00,POINT (-117.19432 32.68559),place
3,319,319,EH-010,2025-03-31,06:21:00,Enterococcus,=,7043.0,Copies/100ml,ddPCR,...,Cortez Ave,"Imperial Beach municipal beach, other",32.572665,-117.133201,2025-04-01 07:15:58,2025-03-31T00:00:00,0 days 06:21:00,2025-03-31T06:21:00,POINT (-117.1332 32.57267),place
4,322,322,EH-030,2025-03-31,06:35:00,Enterococcus,=,5394.0,Copies/100ml,ddPCR,...,Imperial Beach Pier,Imperial Beach pier area,32.578750,-117.133186,2025-04-01 07:15:58,2025-03-31T00:00:00,0 days 06:35:00,2025-03-31T06:35:00,POINT (-117.13319 32.57875),place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2556,322,322,EH-030,2025-01-01,06:45:00,Enterococcus,=,3882.0,Copies/100ml,ddPCR,...,Imperial Beach Pier,Imperial Beach pier area,32.578750,-117.133186,2025-01-03 11:14:32,2025-01-01T00:00:00,0 days 06:45:00,2025-01-01T06:45:00,POINT (-117.13319 32.57875),place
2557,459,459,IB-050,2025-01-01,06:30:00,Enterococcus,=,2635.0,Copies/100ml,ddPCR,...,End of Seacoast Dr,"Imperial Beach municipal beach, other",32.566577,-117.133205,2025-01-03 11:14:32,2025-01-01T00:00:00,0 days 06:30:00,2025-01-01T06:30:00,POINT (-117.13321 32.56658),place
2558,460,460,IB-060,2025-01-01,07:02:00,Enterococcus,=,2236.0,Copies/100ml,ddPCR,...,Carnation Ave.,north Imperial Beach,32.585679,-117.133049,2025-01-03 11:14:32,2025-01-01T00:00:00,0 days 07:02:00,2025-01-01T07:02:00,POINT (-117.13305 32.58568),place
2559,1037,1037,IB-068,2025-01-01,07:20:00,Enterococcus,=,1873.0,Copies/100ml,ddPCR,...,Silver Strand - Guard Shack,Silver Strand State Beach,32.626401,-117.139424,2025-01-03 11:14:32,2025-01-01T00:00:00,0 days 07:20:00,2025-01-01T07:20:00,POINT (-117.13942 32.6264),place


In [5]:
beach_gdf.drop(['date','time', 'id', 'Station_ID'], axis=1).to_csv('../../../../data/beach_water_quality/beachwatch.csv', index=False)

In [8]:
from foursquare.data_sdk import DataSDK, MediaType
import os
refresh_token=os.getenv("RC_FSQ_REFRESH_TOKEN")
#client = DataSDK(refresh_token=refresh_token)
client = DataSDK()

In [9]:
client.upload_dataframe(
            beach_gdf,
          dataset="da36b3ce-10d7-468a-b1c8-152ea2e540db",
            name='Beach Water Quality',
            description="BeachWatch from dataframe")


Compressing file
Uploading: 100%|██████████| 73.2k/73.2k [00:00<00:00, 92.5kB/s]


Dataset(id=UUID('da36b3ce-10d7-468a-b1c8-152ea2e540db'), name='Beach Water Quality', type=<DatasetType.MANAGED: 'managed'>, created_at=datetime.datetime(2025, 2, 13, 17, 45, 17, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2025, 4, 3, 22, 21, 27, tzinfo=TzInfo(UTC)), description='BeachWatch from dataframe', is_valid=True, data_connector=None, metadata=DatasetMetadata(media_type='text/csv', size=703313, source=None, tileset_data_url=None, tileset_metadata_url=None, image_url=None, metadata_url=None, data_status=None))

In [10]:
import datetime
latest_df=beach_gdf[beach_gdf['datetime']>=datetime.datetime(2025, 1, 1)]
client.upload_dataframe(
            latest_df,
           dataset="729ee7c8-ab44-4d0d-90eb-2bcb353d5efa",
            name='Beach Water Quality - Latest',
            description="BeachWatch from dataframe")

TypeError: '>=' not supported between instances of 'str' and 'datetime.datetime'

# CLOUSURES

In [2]:
closures_page= "https://www.sdbeachinfo.com/Home/GetTargetByID"
response = requests.post(closures_page)
data = response.json()
closures_df = pd.DataFrame(data)
gs=gpd.GeoSeries.from_xy(closures_df['Longitude'],closures_df['Latitude'])
closures_gdf=gpd.GeoDataFrame(closures_df, geometry=gs ,crs='EPSG:4326')
closures_gdf['Icon']= 'place'
closures_gdf['RBGColor']= closures_gdf['RBGColor'].map(lambda x: x.replace('.png',''))
closures_gdf

,SiteID,DehID,Name,Latitude,Longitude,CityID,IndicatorID,RegionID,RBGColor,RegionName,...,IndicateDescription,Description,Advisory,Closure,GeneralAdvisoryMessage,Active,StationID,TypeID,geometry,Icon
0,2,IB-010,Border Field State Park - N of fence (IB-010),32.535950,-117.124300,35,1,3,Red,None,...,None,<p><strong>Closure: Tijuana Slough Shoreline</...,"<p><span style=""font-size:14px""><strong>Closur...","<p><span style=""font-size:14px""><strong>Closur...",,True,None,1,POINT (-117.1243 32.53595),place
1,3,IB-020,Border Field State Park at Monument Road (IB-020),32.543410,-117.125228,35,1,3,Red,None,...,None,<p><strong>Closure: Tijuana Slough Shoreline</...,"<p><span style=""font-size:14px""><strong>Closur...","<p><span style=""font-size:14px""><strong>Closur...",,True,None,1,POINT (-117.12523 32.54341),place
2,4,IB-030,Tijuana Slough at Tijuana Rivermouth (IB-030),32.555139,-117.129323,35,1,3,Red,None,...,None,<p><strong>Closure: Tijuana Slough Shoreline</...,"<p><span style=""font-size:14px""><strong>Closur...","<p><span style=""font-size:14px""><strong>Closur...",,True,None,1,POINT (-117.12932 32.55514),place
3,5,IB-040,Tijuana Slough - 3/4mi N of TJ River (IB-040),32.562869,-117.132590,35,1,3,Red,None,...,None,"<p style=""margin-left:0in; margin-right:0in"">T...","<p><span style=""font-size:14px""><strong>Closur...","<p><span style=""font-size:14px""><strong>Closur...",,True,None,1,POINT (-117.13259 32.56287),place
4,6,IB-050,"Seacoast Drive, south - Imperial Beach (IB-050)",32.566577,-117.133205,35,1,3,Red,None,...,None,<p>This beach is sampled year-round (by the Co...,<p><strong>Closure: Imperial Beach Shoreline</...,<p><strong>Closure: Imperial Beach Shoreline</...,,True,None,1,POINT (-117.13321 32.56658),place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,472,MB-015,Enchanted Cove - Mission Bay (MB-015),32.777266,-117.216636,21,2,2,Green,None,...,None,<p>This beach is sampled during the summer sea...,<p><strong>Advisory:</strong> Mission Bay<br /...,<p><strong>Advisory:</strong> Mission Bay<br /...,,True,None,1,POINT (-117.21664 32.77727),place
343,474,SE-060,San Elijo Lagoon outlet (SE-060),33.015700,-117.281912,11,3,1,Yellow,None,...,None,<p>This beach is sampled weekly year-round. Ho...,<p><strong>Advisory</strong>:&nbsp;Cardiff&nbs...,<p><strong>Advisory</strong>:&nbsp;Cardiff&nbs...,,True,None,1,POINT (-117.28191 33.0157),place
344,480,MB-226,Mission Point Park (MB-226),32.760746,-117.247827,21,2,2,Green,None,...,None,<p>This beach is sampled weekly (by the County...,,,,True,None,1,POINT (-117.24783 32.76075),place
345,483,MB-115,South Mission Beach Jetty (MB-115),32.759680,-117.254057,22,2,2,Green,None,...,None,<p>This beach is sampled by the County of San ...,,,,True,None,1,POINT (-117.25406 32.75968),place


In [11]:
client.upload_dataframe(
            closures_gdf,
           dataset="db7c2c8a-6522-47f9-98df-fdc8c7594ac2",
            name='Beach Closures',
            description="Beach Closures from BeachWatch from dataframe")

Compressing file
Uploading: 100%|██████████| 19.2k/19.2k [00:00<00:00, 26.7kB/s]


Dataset(id=UUID('db7c2c8a-6522-47f9-98df-fdc8c7594ac2'), name='Beach Closures', type=<DatasetType.MANAGED: 'managed'>, created_at=datetime.datetime(2025, 3, 20, 23, 30, 10, tzinfo=TzInfo(UTC)), updated_at=datetime.datetime(2025, 3, 20, 23, 55, 43, tzinfo=TzInfo(UTC)), description='Beach Closures from BeachWatch from dataframe', is_valid=True, data_connector=None, metadata=DatasetMetadata(media_type='text/csv', size=207746, source=None, tileset_data_url=None, tileset_metadata_url=None, image_url=None, metadata_url=None, data_status=None))